In [80]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import scipy.ndimage
import cv2
import os
from matplotlib import pyplot as plt
import sys
import skimage
from  scipy import ndimage
# Get the home directory
home_directory = os.path.expanduser('~')

%env SITK_SHOW_COMMAND '/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer'
# Append the parent directory to the Python path
#sys.path.append(os.path.join(home_directory, 'Ashish/UCAN', 'UCAN-PET-CT-image-data-handling-pipeline'))
sys.path.append(os.path.join(home_directory, '/home/andres/VSCode/', 'UCAN-PET-CT-image-data-handling-pipeline'))

from Utils import utils

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

env: SITK_SHOW_COMMAND='/home/andres/Downloads/Slicer-5.4.0-linux-amd64/Slicer'


In [79]:
def get_proj_after_mask(img):
    
    '''

    Function to get 3D masks for CT scans to segment out the exact tissue type.

    '''

    pix_array=sitk.GetArrayFromImage(img)
    max_i, min_i=float(pix_array.max()),float(pix_array.min())

    #multiply= sitk.MultiplyImageFilter()
    #if hu_type == 'Bone' or hu_type == 'bone' or hu_type == 'B':
    bone_mask = sitk.BinaryThreshold(
    img, lowerThreshold=200, upperThreshold=max_i,insideValue=1, outsideValue=0
    )
    
    tracer_contrast_mask = sitk.BinaryThreshold(
    img, lowerThreshold=151, upperThreshold=199,insideValue=1, outsideValue=0
    )

    #bone_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        # path= img_n + r'_{0}_image.nii'.format(modality + '_' + type)
        # save_as_gz(op_img,path)
    #elif hu_type == 'Lean Tissue' or hu_type == 'lean' or hu_type == 'LT':
    lean_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-29, upperThreshold=150, insideValue=1, outsideValue=0
    )
    #lean_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))

    #elif hu_type == 'Adipose' or hu_type == 'adipose' or hu_type == 'AT':
    adipose_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-199, upperThreshold=-30, insideValue=1, outsideValue=0
    )
    #adipose_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        
    #elif hu_type == 'Air' or hu_type == 'A':
    air_mask = sitk.BinaryThreshold(
    img, lowerThreshold=min_i, upperThreshold=-191, insideValue=1, outsideValue=0
    )
    #air_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
    
    return bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask

In [3]:
#CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"
CT_path = "/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"

In [4]:
CT_img = sitk.ReadImage(CT_path)

In [5]:
bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

In [6]:
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
np.unique(lean_mask_arr)

array([0, 1], dtype=uint8)

In [7]:
lean_mask_arr.sum()

2476456

In [8]:
np.unique(lean_mask_arr, return_counts=True)

(array([0, 1], dtype=uint8), array([16332376,  2476456]))

In [9]:
voxel_spacing = [2.0364201068878174, 2.0364201068878174, 3.0]

In [10]:
# calculate lean density of lean muscle # multiply with 1e-3 to covert mm3 to cm3 (ml)  # ml * 1e-3 litres
density = lean_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6s

30.809640000075532

In [21]:
# calculate bone skeleton density
bone_mask_arr = sitk.GetArrayFromImage(bone_mask)
print(np.unique(bone_mask_arr))
print(bone_mask_arr.sum())
print(np.unique(bone_mask_arr, return_counts=True))
bone_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6

[0 1]
262629
(array([0, 1], dtype=uint8), array([18546203,   262629]))


3.2673727873945015

In [36]:
# remove noise from lean mask
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
print(np.unique(lean_mask_arr))
print(lean_mask_arr.sum())
print(lean_mask_arr.shape)
print(np.unique(lean_mask_arr, return_counts=True))

[0 1]
2476456
(287, 256, 256)
(array([0, 1], dtype=uint8), array([16332376,  2476456]))


In [112]:
temp = lean_mask_arr.copy()
lean_mask_arr1 = ndimage.binary_fill_holes(temp, structure=np.ones((3,3,3)) ).astype(int)
print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1)
lean_mask_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)#, CT_img.GetPixelID())

[0 1]
16348356
(287, 256, 256)
(array([0, 1]), array([ 2460476, 16348356]))


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-37.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-37.mha" "[0.30s]"


In [146]:
temp = 1 - lean_mask_arr
#temp = lean_mask_arr.copy()
#temp = skimage.measure.label(temp)
lean_mask_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
lean_mask_arr1 = skimage.morphology.remove_small_objects(lean_mask_arr1.astype(bool),min_size=500)
#lean_mask_arr1 = skimage.morphology.remove_small_holes(lean_mask_arr1)

print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1.astype(int))
print(lean_mask_img1.GetSize())
lean_mask_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)

[False  True]
1542029
(287, 256, 256)
(array([False,  True]), array([17266803,  1542029]))
(256, 256, 287)


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-69.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-69.mha" "[0.30s]"


In [151]:
tracer_contrast_arr = sitk.GetArrayFromImage(tracer_contrast_mask)
temp = 1 - tracer_contrast_arr
#temp = lean_mask_arr.copy()
#temp = skimage.measure.label(temp)
#tracer_contrast_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
#tracer_contrast_arr1 = skimage.morphology.remove_small_objects(tracer_contrast_arr1.astype(bool),min_size=200)
#lean_mask_arr1 = skimage.morphology.remove_small_holes(lean_mask_arr1)
tracer_contrast_arr1 = tracer_contrast_arr.copy()
temp = 1 - lean_mask_arr
#temp = lean_mask_arr.copy()
#temp = skimage.measure.label(temp)
lean_mask_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
lean_mask_arr1 = skimage.morphology.remove_small_objects(lean_mask_arr1.astype(bool),min_size=500)
#lean_mask_arr1 = skimage.morphology.remove_small_holes(lean_mask_arr1)

print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1.astype(int))
print(lean_mask_img1.GetSize())
lean_mask_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)
print(np.unique(tracer_contrast_arr1))
print(tracer_contrast_arr1.sum())
print(tracer_contrast_arr1.shape)
print(np.unique(tracer_contrast_arr1, return_counts=True))
tracer_contrast_img1 = sitk.GetImageFromArray(tracer_contrast_arr1.astype(int))
print(tracer_contrast_img1.GetSize())
tracer_contrast_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(tracer_contrast_img1)

[0 1]
92344
(287, 256, 256)
(array([0, 1], dtype=uint8), array([18716488,    92344]))
(256, 256, 287)


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-74.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-74.mha" "[0.28s]"


In [116]:
temp = 1 - lean_mask_arr
#temp = lean_mask_arr.copy()
#lean_mask_arr1 = skimage.morphology.remove_small_objects(temp, min_size=1500, connectivity=124, out=temp)
#footprint = skimage.morphology.octahedron(1)
#footprint = skimage.morphology.cube(5)
#lean_mask_arr1 = skimage.morphology.white_tophat(temp, footprint)
lean_mask_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
#lean_mask_arr1 = skimage.morphology.binary_erosion(lean_mask_arr1).astype(int)
#lean_mask_arr1 = skimage.morphology.binary_erosion(lean_mask_arr1, footprint=[(np.ones((3, 3, 3)), 2)]).astype(int)
print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1.astype(int))
print(lean_mask_img1.GetSize())
lean_mask_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)#, CT_img.GetPixelID())

[0 1]
1564318
(287, 256, 256)
(array([0, 1]), array([17244514,  1564318]))
(256, 256, 287)


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-40.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-40.mha" "[0.28s]"


In [105]:
CT_img.GetSize()

(256, 256, 287)

In [147]:
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/"
multiply= sitk.MultiplyImageFilter()
CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(CT_lean)
utils.get_2D_projections(CT_lean, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MaxIP2_')
utils.get_2D_projections(CT_lean, 'CT', "min", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MinIP2_')


Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-3168-70.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-3168-70.mha" "[0.15s]"


In [13]:
multiply= sitk.MultiplyImageFilter()
CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))

CT_ptype = 'mean'
SUV_ptype = 'max'
angle = 90

#raw_projections_path = "/home/ashish/Ashish/UCAN/Testing/test_new_projections/"
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_new_projections/"

utils.get_2D_projections(CT_tracer_contrast, 'CT', CT_ptype, angle, invert_intensity= True, img_n=raw_projections_path)

Finished generating 3 - mean intensity 2D projections from the CT volume image! 


In [ ]:
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/"
multiply= sitk.MultiplyImageFilter()
CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(CT_lean)
utils.get_2D_projections(CT_lean, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MaxIP2_')
utils.get_2D_projections(CT_lean, 'CT', "min", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MinIP2_')


In [27]:
resampled_directory_list = []
#resampled_SUV_CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/"
resampled_SUV_CT_path = "/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/"

for dirs, subdirs, files in os.walk(resampled_SUV_CT_path):
    for file in files:
        file_path = str(os.path.join(dirs, file))
        file_path = file_path.replace('\\','/')
        resampled_directory_list.append(file_path)

resampled_directory_df = pd.DataFrame(resampled_directory_list, columns=['directory'])
resampled_directory_df[['source_directory', 'patient_directory', 'scan_date', 'SUV_CT']] = resampled_directory_df['directory'].str.rsplit(pat='/', n=3, expand=True)
resampled_directory_df[['npr', 'extra']] = resampled_directory_df['patient_directory'].str.split(pat='_', n=1, expand=True)
resampled_directory_df.drop(columns=['directory','extra', 'SUV_CT'], inplace=True)
resampled_directory_df.drop_duplicates(inplace=True)

utils.display_full(resampled_directory_df.head(2))

count = 0
# Generate the raw 2D projections
for index, row in resampled_directory_df.iterrows():
    #raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"
    raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_new_projections/"
    angle = 90

    CTnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'CT.nii.gz'
    #print("CTnii_path", CTnii_path)
    
    CT_img =sitk.ReadImage(CTnii_path)
    #print("info:", CT_img.GetSize())
    #utils.save_as_gz(CT_img, raw_projections_path + 'test_CT.nii.gz')
    #sitk.WriteImage(CT_img, raw_projections_path + 'test_CT.nii.gz')

    SUVnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'SUV.nii.gz'
    SUV_img =sitk.ReadImage(SUVnii_path)
    #print("SUVnii_path", SUVnii_path)
    #utils.save_as_gz(SUV_img, raw_projections_path + 'test_SUV.nii.gz')
    
    bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

    multiply= sitk.MultiplyImageFilter()
    #try:

    CT_tracer_contrast = multiply.Execute(CT_img,sitk.Cast(tracer_contrast_mask,CT_img.GetPixelID()))
 
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MaxIP_')
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(CT_tracer_contrast, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MinIP_')

    SUV_tracer_contrast = multiply.Execute(SUV_img,sitk.Cast(tracer_contrast_mask,SUV_img.GetPixelID()))
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MaxIP_')
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MinIP_')


    CT_bone = multiply.Execute(CT_img,sitk.Cast(bone_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_bone, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MaxIP_')
    #utils.get_2D_projections(CT_bone, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MinIP_')

    CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask,CT_img.GetPixelID()))
    utils.get_2D_projections(CT_lean, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MaxIP_')
    utils.get_2D_projections(CT_lean, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MinIP_')
    
    count += 1
    print(str(row['npr']) + '_' + str(row['scan_date']))
    print("count: ", count)
    #except: 
        #print(str(row['npr']) + '_' + str(row['scan_date']))
    #    pass

    if count > 10:
        break

                                         source_directory       patient_directory scan_date              npr
0  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr385705046400_SUV_CT  20140313  lpr385705046400
2  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr415675513429_SUV_CT  20190201  lpr415675513429
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the SUV volume image! 
Finished generating 3 - mean intensity 2D projections from the SUV volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 
lpr385705046400_20140313
count:  1
Finished generating 3 - max intensity 2D projections from the CT volume im

In [18]:
resampled_directory_df.head()

,source_directory,patient_directory,scan_date,npr
0,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20210622,npr865130139443
2,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr865130139443_SUV_CT,20211126,npr865130139443
4,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20150127,npr382894159583
6,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr382894159583_SUV_CT,20140828,npr382894159583
8,/home/ashish/Ashish/UCAN/Resampled_SUV_CT,npr717322658733_SUV_CT,20180718,npr717322658733


### Check body mask

In [13]:
temp = 1 - lean_mask_arr
#temp = lean_mask_arr.copy()
#temp = skimage.measure.label(temp)
lean_mask_arr1 = skimage.morphology.binary_erosion(temp).astype(int)
lean_mask_arr1 = skimage.morphology.remove_small_objects(lean_mask_arr1.astype(bool),min_size=500)
#lean_mask_arr1 = skimage.morphology.remove_small_holes(lean_mask_arr1)

print(np.unique(lean_mask_arr1))
print(lean_mask_arr1.sum())
print(lean_mask_arr1.shape)
print(np.unique(lean_mask_arr1, return_counts=True))
lean_mask_img1 = sitk.GetImageFromArray(lean_mask_arr1.astype(int))
print(lean_mask_img1.GetSize())
lean_mask_img1.CopyInformation(CT_img)
#multiply= sitk.MultiplyImageFilter()
#CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(lean_mask_img1)

[False  True]
15239736
(287, 256, 256)
(array([False,  True]), array([ 3569096, 15239736]))
(256, 256, 287)


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-0.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-0.mha" "[0.29s]"


In [ ]:
npr100589333593 20190205

In [22]:
def fill_mask_array(imgs, selem = None):

    """ Fill binary holes, applied slicewise """

    mask = np.zeros(imgs.shape)

    for s in range(imgs.shape[0]):

        if selem is not None:
            mask[s] = ndimage.binary_fill_holes(imgs[s], selem)

        else:
            mask[s] = ndimage.binary_fill_holes(imgs[s])

    return mask

In [101]:
#CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr100589333593_SUV_CT/20190205/CT.nii.gz")

#CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr105542510597_SUV_CT/20181011/CT.nii.gz")

#CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr110222274795_SUV_CT/20210330/CT.nii.gz")

#CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr150091528838_SUV_CT/20201023/CT.nii.gz")

CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr100222372477_SUV_CT/20171207/CT.nii.gz")

#CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/npr110222274795_SUV_CT/20210330/CT.nii.gz")

#sitk.Show(CT_img)

In [102]:
CT_array = sitk.GetArrayFromImage(CT_img)
#bodymask_guess = skimage.morphology.binary_opening(fill_mask_array(np.where(CT_array > -500, 1.0, 0)), skimage.morphology.ball(5))

#bodymask_arr = skimage.morphology.binary_opening(np.where(CT_array > -500, 1.0, 0), skimage.morphology.ball(5))
#bodymask_arr = skimage.morphology.binary_closing(bodymask_arr, skimage.morphology.ball(5))

bodymask_arr = skimage.morphology.binary_opening(fill_mask_array(np.where(CT_array > -100, 1.0, 0)), skimage.morphology.ball(5))
#bodymask_arr = skimage.morphology.binary_opening(bodymask_arr, skimage.morphology.ball(5))
print(bodymask_arr.shape)
bodymask_arr = fill_mask_array(bodymask_arr)
bodymask_arr = fill_mask_array(np.transpose(bodymask_arr, (1,2,0)))
bodymask_arr = fill_mask_array(np.transpose(bodymask_arr, (1,2,0)))

#print(bodymask_arr.shape)
#bodymask_arr = fill_mask_array(np.transpose(bodymask_arr, (1,2,0)))
#print(bodymask_arr.shape)
#bodymask_arr = ndimage.binary_fill_holes(bodymask_arr.astype(bool), structure=np.ones((5,5,5)) ).astype(int)bodymask_arr
#bodymask_arr = ndimage.binary_fill_holes(bodymask_arr.astype(bool), skimage.morphology.ball(6)).astype(int)
#bodymask_arr = skimage.morphology.binary_dilation(bodymask_arr,  skimage.morphology.ball(5))

#bodymask_arr = 1 - bodymask_arrnp.where(CT_array > -500, 1.0, 0)), structure=np.ones((8,8,8)))
#bodymask_arr = 1 - bodymask_arr
bodymask_arr = np.transpose(bodymask_arr, (1,2,0))

print(bodymask_arr.shape)                
print(np.unique(bodymask_arr))
print(bodymask_arr.sum())
print(np.unique(bodymask_arr, return_counts=True))
bodymask_img = sitk.GetImageFromArray(bodymask_arr.astype(int))
print(bodymask_img.GetSize())
bodymask_img.CopyInformation(CT_img)
sitk.Show(bodymask_img)

(207, 256, 256)
(207, 256, 256)
[0. 1.]
2984567.0
(array([0., 1.]), array([10581385,  2984567]))
(256, 256, 207)


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-61.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-61.mha" "[0.17s]"
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""
Switch to module:  ""


In [74]:
multiply= sitk.MultiplyImageFilter()
CT_img_onlybody = multiply.Execute(CT_img,sitk.Cast(bodymask_img,CT_img.GetPixelID()))
sitk.Show(CT_img_onlybody)

Switch to module:  "Welcome"
Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-44.mha"
Local filepath received via command-line:  "/tmp/TempFile-28229-43.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-44.mha" "[0.12s]"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-43.mha" "[0.23s]"


In [36]:
raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/27Dec2023/"
utils.get_2D_projections(CT_img_onlybody, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + "npr100589333593_20190205" + '_CT_MaxIP_')
utils.get_2D_projections(CT_img_onlybody, 'CT', "mean", angle=90, invert_intensity= False, img_n=raw_projections_path + "npr100589333593_20190205" + '_CT_MeanIP_')

Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 


In [37]:
background_mask = np.where(bodymask_arr == 1, 0, -1024)


multiply= sitk.MultiplyImageFilter()
CT_img_onlybody = multiply.Execute(CT_img,sitk.Cast(bodymask_img,CT_img.GetPixelID()))
sitk.Show(CT_img_onlybody)

CT_img_onlybody_arr = sitk.GetArrayFromImage(CT_img_onlybody)

final_onlybody_image = sitk.GetImageFromArray(CT_img_onlybody_arr + background_mask)

raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/27Dec2023_v2/"
utils.get_2D_projections(final_onlybody_image, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + "npr100589333593_20190205" + '_CT_MaxIP_')
utils.get_2D_projections(final_onlybody_image, 'CT', "mean", angle=90, invert_intensity= False, img_n=raw_projections_path + "npr100589333593_20190205" + '_CT_MeanIP_')

Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-9.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-9.mha" "[0.17s]"


In [15]:
#/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm"
CT_img = sitk.ReadImage("/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/lpr385705046400_SUV_CT/20140313/CT.nii.gz")
CT_array = sitk.GetArrayFromImage(CT_img)
bodymask_arr = skimage.morphology.binary_opening(np.where(CT_array > -500, 1.0, 0), skimage.morphology.ball(6))
bodymask_arr = skimage.morphology.binary_closing(bodymask_arr, skimage.morphology.ball(6))
bodymask_img = sitk.GetImageFromArray(bodymask_arr.astype(int))
#print(bodymask_img.GetSize())
bodymask_img.CopyInformation(CT_img)
multiply= sitk.MultiplyImageFilter()
CT_img_onlybody = multiply.Execute(CT_img,sitk.Cast(bodymask_img,CT_img.GetPixelID()))
bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

sitk.Show(CT_img_onlybody)

Local filepath received via command-line:  "/tmp/TempFile-28229-1.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-1.mha" "[0.33s]"


Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-2.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-2.mha" "[0.14s]"


In [16]:
sitk.Show(CT_img)

MetaImageIO (0x33d9290): Unsupported or empty metaData item ITK_FileNotes of type Ssfound, won't be written to image file

MetaImageIO (0x33d9290): Unsupported or empty metaData item aux_file of type Ssfound, won't be written to image file

MetaImageIO (0x33d9290): Unsupported or empty metaData item descrip of type Ssfound, won't be written to image file

MetaImageIO (0x33d9290): Unsupported or empty metaData item intent_name of type Ssfound, won't be written to image file

MetaImageIO (0x33d9290): Unsupported or empty metaData item qto_xyz of type N3itk6MatrixIfLj4ELj4EEEfound, won't be written to image file



Switch to module:  "Welcome"
Local filepath received via command-line:  "/tmp/TempFile-28229-3.mha"
"Volume" Reader has successfully read the file "/tmp/TempFile-28229-3.mha" "[0.15s]"


In [ ]:


raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/22Dec2023/"
multiply= sitk.MultiplyImageFilter()
CT_lean = multiply.Execute(CT_img,sitk.Cast(lean_mask_img1,CT_img.GetPixelID()))
sitk.Show(CT_lean)
utils.get_2D_projections(CT_lean, 'CT', "max", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MaxIP2_')
utils.get_2D_projections(CT_lean, 'CT', "min", angle=90, invert_intensity= True, img_n=raw_projections_path + 'npr100169878450_SUV_CT_20130412_CT_lean_MinIP2_')


In [181]:
resampled_directory_list = []
#resampled_SUV_CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/"
resampled_SUV_CT_path = "/media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT/"

for dirs, subdirs, files in os.walk(resampled_SUV_CT_path):
    for file in files:
        file_path = str(os.path.join(dirs, file))
        file_path = file_path.replace('\\','/')
        resampled_directory_list.append(file_path)

resampled_directory_df = pd.DataFrame(resampled_directory_list, columns=['directory'])
resampled_directory_df[['source_directory', 'patient_directory', 'scan_date', 'SUV_CT']] = resampled_directory_df['directory'].str.rsplit(pat='/', n=3, expand=True)
resampled_directory_df[['npr', 'extra']] = resampled_directory_df['patient_directory'].str.split(pat='_', n=1, expand=True)
resampled_directory_df.drop(columns=['directory','extra', 'SUV_CT'], inplace=True)
resampled_directory_df.drop_duplicates(inplace=True)

utils.display_full(resampled_directory_df.head(2))

count = 0
# Generate the raw 2D projections
for index, row in resampled_directory_df.iterrows():
    #raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"
    raw_projections_path = "/media/andres/T7 Shield1/UCAN_project/Testing/test_lean_projection_after_removing_noise/22Dec2023/"
    angle = 90

    CTnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'CT.nii.gz'
    #print("CTnii_path", CTnii_path)
    
    CT_img =sitk.ReadImage(CTnii_path)
    #print("info:", CT_img.GetSize())
    #utils.save_as_gz(CT_img, raw_projections_path + 'test_CT.nii.gz')
    #sitk.WriteImage(CT_img, raw_projections_path + 'test_CT.nii.gz')

    SUVnii_path = resampled_SUV_CT_path + str(row["patient_directory"]) + '/' + str(row['scan_date']) + '/' + 'SUV.nii.gz'
    SUV_img =sitk.ReadImage(SUVnii_path)
    #print("SUVnii_path", SUVnii_path)
    #utils.save_as_gz(SUV_img, raw_projections_path + 'test_SUV.nii.gz')

    #bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img)

    #Extracting only body from CT
    CT_array = sitk.GetArrayFromImage(CT_img)
    bodymask_arr = skimage.morphology.binary_opening(np.where(CT_array > -500, 1.0, 0), skimage.morphology.ball(6))
    bodymask_arr = skimage.morphology.binary_closing(bodymask_arr, skimage.morphology.ball(6))
    bodymask_img = sitk.GetImageFromArray(bodymask_arr.astype(int))
    bodymask_img.CopyInformation(CT_img)
    multiply= sitk.MultiplyImageFilter()
    CT_img_onlybody = multiply.Execute(CT_img,sitk.Cast(bodymask_img,CT_img.GetPixelID()))
    bone_mask, lean_mask, adipose_mask, air_mask, tracer_contrast_mask = get_proj_after_mask(CT_img_onlybody)

    multiply= sitk.MultiplyImageFilter()
    #try:

    CT_tracer_contrast = multiply.Execute(CT_img_onlybody,sitk.Cast(tracer_contrast_mask,CT_img_onlybody.GetPixelID()))
 
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MaxIP_')
    utils.get_2D_projections(CT_tracer_contrast, 'CT', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(CT_tracer_contrast, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_tracer_contrast_MinIP_')

    SUV_tracer_contrast = multiply.Execute(SUV_img,sitk.Cast(tracer_contrast_mask,SUV_img.GetPixelID()))
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MaxIP_')
    utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "mean", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MeanIP_')
    #utils.get_2D_projections(SUV_tracer_contrast, 'SUV', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_SUV_tracer_contrast_MinIP_')


    CT_bone = multiply.Execute(CT_img_onlybody,sitk.Cast(bone_mask,CT_img_onlybody.GetPixelID()))
    utils.get_2D_projections(CT_bone, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MaxIP_')
    utils.get_2D_projections(CT_bone, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MinIP_')

    CT_lean = multiply.Execute(CT_img_onlybody,sitk.Cast(lean_mask,CT_img_onlybody.GetPixelID()))
    utils.get_2D_projections(CT_lean, 'CT', "max", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MaxIP_')
    utils.get_2D_projections(CT_lean, 'CT', "min", angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_lean_MinIP_')
    
    count += 1
    print(str(row['npr']) + '_' + str(row['scan_date']))
    print("count: ", count)
    #except: 
        #print(str(row['npr']) + '_' + str(row['scan_date']))
    #    pass

    if count > 10:
        break

                                         source_directory       patient_directory scan_date              npr
0  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr385705046400_SUV_CT  20140313  lpr385705046400
2  /media/andres/T7 Shield1/UCAN_project/Resampled_SUV_CT  lpr415675513429_SUV_CT  20190201  lpr415675513429
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the SUV volume image! 
Finished generating 3 - mean intensity 2D projections from the SUV volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 


/home/andres/VSCode/UCAN-PET-CT-image-data-handling-pipeline/Utils/utils.py:99: RuntimeWarning: invalid value encountered in divide
  arr_normed = (arr - minv) / (maxv - minv)


Finished generating 3 - min intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 
lpr385705046400_20140313
count:  1
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - mean intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the SUV volume image! 
Finished generating 3 - mean intensity 2D projections from the SUV volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - min intensity 2D projections from the CT volume image! 
lpr415675513429_20190201
count:  2
Finished generating 3 - max intensity 2D proje